# Setup et imports

In [ ]:
import sys
sys.path.append('../src')  # Ajoute le dossier src au path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from notebook_helpers  import load_all_data, quick_audit
from config import DB_PARAMS, logger


# Chargement des Données

In [ ]:
# Charger toutes les données
df_anime, df_genres, df_studios = load_all_data()


# Audit Rapide (Data Quality)

In [ ]:
# Audit rapide
quick_audit(df_anime, "Animes")

# Première Analyse - Distribution des Scores

In [ ]:
# Histogramme des scores
plt.figure(figsize=(10, 6))
# On enlève les NaN pour le graphique
df_anime['score'].dropna().hist(bins=20, edgecolor='black')
plt.title('Distribution des Scores des Animes')
plt.xlabel('Score (sur 100)')
plt.ylabel("Nombre d'animes")
plt.grid(False) # Pour un look plus propre parfois
plt.show()

# Top 10 des Genres les plus populaires

In [ ]:
# Compter le nombre d'occurrences de chaque genre
top_genres = df_genres['genre'].value_counts().head(15)

# Graphique en barres
plt.figure(figsize=(10, 6))
top_genres.plot(kind='bar', color='skyblue')
plt.title('Top 10 des Genres les plus fréquents')
plt.ylabel("Nombre d'animes")
plt.xticks(rotation=45)
plt.show()

# Quels sont les studios les plus prolifiques ?

In [ ]:
# Compter les animes par studio
top_studios = df_studios['studio_name'].value_counts().head(10)

# Visualiser
plt.figure(figsize=(10, 6))
top_studios.sort_values().plot(kind='barh', color='lightgreen') # barh pour des barres horizontales
plt.title('Top 10 Studios par nombre d\'animes produits')
plt.xlabel("Nombre d'animes")
plt.show()

# Analyse : Studio vs Score

In [ ]:
# Merger avec les infos des animes (pour avoir le score)
df_studio_score = pd.merge(df_studios, df_anime[['anime_id', 'score']], on='anime_id')

# Filtrer les scores non nuls
df_studio_score = df_studio_score[df_studio_score['score'].notna()]

# Calculer le score moyen de tous les studios
all_studio_stats = df_studio_score.groupby('studio_name').agg({
    'score': 'mean',
    'anime_id': 'count'
}).round(1)
all_studio_stats.columns = ['Score Moyen', 'Nombre d\'animes']

# Filtrer : au moins 5 animes pour éviter les studios avec 1 seul anime à 90
studios_qualified = all_studio_stats[all_studio_stats['Nombre d\'animes'] >= 9]

# Top 15 par score moyen
studio_stats = studios_qualified.sort_values('Score Moyen', ascending=False).head(30)

print(studio_stats)

# Visualisation
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 10))

# Graphique 1 : Score moyen par studio
studio_stats['Score Moyen'].sort_values().plot(kind='barh', ax=ax1, color='coral')
ax1.set_title('Score moyen par studio (Top 30)', fontsize=12, fontweight='bold')
ax1.set_xlabel('Score moyen', fontsize=10)
ax1.grid(axis='x', alpha=0.3)

# Graphique 2 : Nombre d'animes vs Score moyen (scatter)
ax2.scatter(studio_stats['Nombre d\'animes'], studio_stats['Score Moyen'], 
            s=100, alpha=0.6, color='steelblue')
ax2.set_title('Productivité vs Qualité', fontsize=12, fontweight='bold')
ax2.set_xlabel('Nombre d\'animes produits', fontsize=10)
ax2.set_ylabel('Score moyen', fontsize=10)
ax2.grid(alpha=0.3)

# Annoter les points
for studio, row in studio_stats.iterrows():
    ax2.annotate(studio, (row['Nombre d\'animes'], row['Score Moyen']), 
                 fontsize=8, alpha=0.7)

plt.tight_layout()
plt.show()

# Heatmap : Genre vs Décennie

In [ ]:
# On prépare les données : on joint animes et genres
df_merged = pd.merge(df_anime, df_genres, on='anime_id')

# On crée la décennie
df_merged['decade'] = (df_merged['start_year'] // 10 * 10).astype('Int64')

# On filtre pour ne garder que les décennies à partir de 1960
df_merged = df_merged[df_merged['decade'] >= 1960]

# On crée une table pivot : Lignes = Genres, Colonnes = Décennies, Valeurs = Nombre d'animes
# On ne garde que les genres principaux pour que ce soit lisible
top_genres_list = df_genres['genre'].value_counts().head(10).index
df_heatmap = df_merged[df_merged['genre'].isin(top_genres_list)]

pivot_table = df_heatmap.pivot_table(
    index='genre', columns='decade', values='anime_id', aggfunc='count', fill_value=0
)

# Calculer les totaux par décennie (animes uniques)
total_per_decade = df_merged.groupby('decade')['anime_id'].nunique()

# Normalisation par colonne (par année)
pivot_normalized = pivot_table.div(pivot_table.sum(axis=0), axis=1)



#  Créer 2 sous-graphiques
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), gridspec_kw={'height_ratios': [1, 4]})

# Barplot identique
ax1.bar(total_per_decade.index, total_per_decade.values, color='steelblue', edgecolor='black', alpha=0.8)
ax1.set_title("Nombre total d'animes produits par décennie", fontsize=12, fontweight='bold')
ax1.set_ylabel("Nombre d'animes", fontsize=10)
ax1.set_xticks(total_per_decade.index)
ax1.grid(axis='y', alpha=0.3)
for decade, count in total_per_decade.items():
    ax1.text(decade, count + 50, str(count), ha='center', va='bottom', fontweight='bold')

# Heatmap en %
sns.heatmap(
    pivot_normalized,
    annot=True,
    fmt='.2f',
    cmap='YlOrRd',
    linewidths=0.5,
    cbar_kws={'label': 'Part de marché'},
    ax=ax2
)
ax2.set_title('Part de marché des genres par décennie', fontsize=12, fontweight='bold')
ax2.set_xlabel('Décennie', fontsize=11)
ax2.set_ylabel('Genre', fontsize=11)

plt.tight_layout()
plt.show()

plt.tight_layout()
plt.show()